In [ ]:
# ! pip install ../
# ! pip install matplotlib
# ! pip install langchain
# ! pip install openai

Functionality presented below was first implemented as a Chain in langchain and can be found [here](https://github.com/langchain-ai/langchain/blob/master/cookbook/learned_prompt_optimization.ipynb)

LLM prompts can be enhanced by injecting specific terms into template sentences. Selecting the right terms is crucial for obtaining high-quality responses. This notebook introduces automated prompt engineering through term injection using `learn_to_pick.PickBest`.

For illustration, consider the scenario of a meal delivery service. We want to ask customers, like Tom, about their dietary preferences and recommend suitable meals from our extensive menu. The picker selects a meal based on user preferences, injects it into a prompt template, and forwards the prompt to an LLM. The LLM's response, which is a personalized recommendation, is then returned to the user.

The example laid out below is a toy example to demonstrate the applicability of the concept.

In [ ]:
# four meals defined, some vegetarian some not

meals = [
    "Beef Enchiladas with Feta cheese. Mexican-Greek fusion",
    "Chicken Flatbreads with red sauce. Italian-Mexican fusion",
    "Veggie sweet potato quesadillas with vegan cheese",
    "One-Pan Tortelonni bake with peppers and onions",
]

In [ ]:
# pick and configure the LLM of your choice

from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    temperature=0,
    request_timeout=10,
    max_retries=3,
    client=None,
)

llm.predict("hey")

##### Lets define our prompt that will be personalized

The prompt template which will be used to personalize the message using the LLM call needs to be defined.
It can be anything, but here `{meal}` is being used and is going to be replaced by one of the meals above, the picker will try to pick and inject the best meal


In [ ]:
from langchain.prompts import PromptTemplate

# here I am using the variable meal which will be replaced by one of the meals above
# and some variables like user, preference, and text_to_personalize which I will provide at chain run time

PROMPT_TEMPLATE = """Here is the description of a meal: "{meal}".

Embed the meal into the given text: "{text_to_personalize}".

Prepend a personalized message including the user's name "{user}" 
    and their preference "{preference}".

Make it sound good. Do NOT change the description of the meal.
"""

PROMPT = PromptTemplate(
    input_variables=["meal", "text_to_personalize", "user", "preference"], 
    template=PROMPT_TEMPLATE
)

# create the llm chain and initialize it with the prompt
from langchain import  LLMChain
llm_for_text_generation = LLMChain(prompt=PROMPT, llm=llm)


In [ ]:
import learn_to_pick

# the llm provided here is going to serve as the scoring llm
picker = learn_to_pick.PickBest.create(llm=llm)


In [ ]:
response = picker.run(
    meal = learn_to_pick.ToSelectFrom(meals),
    user = learn_to_pick.BasedOn("Tom"),
    preference = learn_to_pick.BasedOn(["Vegetarian", "regular dairy is ok"]),
)

picked_meal = response["picked"][0]["meal"]

In [54]:
print(picked_meal)

Beef Enchiladas with Feta cheese. Mexican-Greek fusion


In [56]:
llm_for_text_generation.run(meal = picked_meal, user = "Tom", preference = "Vegetarian", text_to_personalize = "This is the weeks specialty dish, our master chefs believe you will love it!")

"Hey Tom! We know you're a Vegetarian, but we think you might be tempted by this week's specialty dish. Our master chefs have created a delicious Mexican-Greek fusion: Beef Enchiladas with Feta cheese. They believe you will absolutely love it!"

#### Alternative example

Below is an alternative way to achieve the same result.

Additionally it showcases how the llm response can be used in the scorer to determine the quality of the decision.

The picker will:
- make a selection using the decision making policy
- call the scorer to evaluate the decision
- update the decision making policy with the score

Callback functions can be registered at create time and they will be called after the decision has been made and before the scorer is called.

In [59]:
# Example of setting own AutoSelectionScorer prompt

# the below scoring prompt wants to use the llm_response to determine whether the meal is good or bad

from langchain.prompts.prompt import PromptTemplate
import langchain

REWARD_PROMPT_TEMPLATE = """

Given preference: "{preference}" rank how good or bad this selection is selection: "{meal}" using the full text to see if it fits: "{llm_response}"

IMPORANT: you MUST return a single number between -1 and 1, -1 being bad, 1 being good

"""


REWARD_PROMPT = PromptTemplate(
    input_variables=["preference", "meal", "llm_response"],
    template=REWARD_PROMPT_TEMPLATE,
)

Because the above custom reward prompt wants to use the llm text generated response to determine whether the selection was good or bad, we can register a custom function that will call the `llm_for_text_generation` and set the response in the inputs. The callback will be called before scoring, so the custom scorer will have access to the generated text.

In [60]:
class CallbackClass:
    def __init__(self, llm):
        self.llm = llm

    def set_llm_response(self, inputs, picked, event):
        print(f"from callback function, here are the inputs: {inputs}")
        
        response = self.llm.predict(**inputs)
        print(f"response from llm: {response}")
        inputs.update({"llm_response": response})
        return inputs, picked, event


In [62]:
the_callback_class = CallbackClass(llm=llm_for_text_generation)

chain = learn_to_pick.PickBest.create(
    callbacks_before_scoring = [the_callback_class.set_llm_response],
    selection_scorer=learn_to_pick.AutoSelectionScorer(llm=llm, prompt=REWARD_PROMPT),
)

response = chain.run(
    meal = learn_to_pick.ToSelectFrom(meals),
    user = learn_to_pick.BasedOn("Tom"),
    preference = learn_to_pick.BasedOn(["Vegetarian", "regular dairy is ok"]),
    text_to_personalize = "This is the weeks specialty dish, our master chefs believe you will love it!",
)

from callback function, here are the inputs: {'meal': 'Veggie sweet potato quesadillas with vegan cheese', 'user': Tom, 'preference': ['Vegetarian', 'regular dairy is ok'], 'text_to_personalize': 'This is the weeks specialty dish, our master chefs believe you will love it!', 'rl_chain_selected_based_on': "{'user': ['Tom'], 'preference': ['Vegetarian', 'regular dairy is ok']}", 'rl_chain_selected': 'Veggie sweet potato quesadillas with vegan cheese'}
response from llm: Hey Tom! As a Vegetarian who's okay with regular dairy, we've got the perfect dish for you this week. Introducing our specialty dish: Veggie sweet potato quesadillas with vegan cheese! Our master chefs believe you will absolutely love this delightful and satisfying meal. Don't miss out on this delicious treat!


In [66]:
# we can now get the llm response from here
response["picked_metadata"].outputs["llm_response"]

"Hey Tom! As a Vegetarian who's okay with regular dairy, we've got the perfect dish for you this week. Introducing our specialty dish: Veggie sweet potato quesadillas with vegan cheese! Our master chefs believe you will absolutely love this delightful and satisfying meal. Don't miss out on this delicious treat!"

In [65]:
# if we want to see what the score was, that was set from the auto selection scorer
response["picked_metadata"].selected.score

1.0